In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#! pip install sentence-transformers

In [ ]:
import os
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel
from sentence_transformers import SentenceTransformer
import torch
import pickle
from tqdm import tqdm
import numpy as np

In [ ]:
data_dir = '/content/gdrive/MyDrive/MLDS/Data_Mining/Project/data/final_data'
os.listdir(data_dir)

['testset.csv',
 'trainset_rec.csv',
 'process_user_business_data_v1.json',
 'testset_ind.csv',
 'trainset_rec_ind.csv',
 'train_tfidf_matrix.npz',
 'tfidf_vectorizer.pkl',
 'svd_500_tfidf_matrix.npy',
 'train_data.json',
 'test_data.json',
 'user_profiles.pkl',
 'item_profiles.pkl',
 'user_item_matrix.pkl',
 'user_sim_matrix.pkl']

In [ ]:
train_fname = 'train_data.json'
train_df = pd.read_json(os.path.join(data_dir,train_fname))
train_df.head(2)

,review_id,user_id,business_id,user_rating,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,item_rating,review_count,is_open,attributes,categories,hours
0,LEGT4hPIyEVMzy4HUROtoQ,-1MF2tosrw2WcCxeVNk81Q,9c7MUiE6VI8NesjPdj5FkA,3,4,0,1,A coworker and I stopped by here for an aftern...,2017-12-13 22:58:16,Bubblefish,...,PA,19107,39.953679,-75.155108,4.0,544,1,"{'RestaurantsReservations': 'True', 'Caters': ...","Restaurants, Sushi Bars, Japanese, Bubble Tea,...","{'Monday': '11:0-23:30', 'Tuesday': '11:0-23:3..."
1,QvoAHbml7mkqFrx9K18h2Q,-1MF2tosrw2WcCxeVNk81Q,1OfhM-ZKvcpxyxptCCzEwA,4,4,0,0,Stopped by here for happy hour and was fairly ...,2017-08-01 22:47:11,Independence Beer Garden,...,PA,19106,39.950294,-75.150981,3.5,513,1,"{'HappyHour': 'True', 'RestaurantsTableService...","Bars, Nightlife, American (New), Gastropubs, F...","{'Monday': '0:0-0:0', 'Wednesday': '15:0-0:0',..."


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device=device)

cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Encoding text to embeddings
review_list = train_df.text.tolist()
encoded_reviews = model.encode(review_list, show_progress_bar=True)

Batches:   0%|          | 0/7515 [00:00<?, ?it/s]

In [ ]:
with open(os.path.join(data_dir, 'bert_embeddings.pkl'), 'wb') as f:
  pickle.dump(encoded_reviews, f)

In [ ]:
encoded_reviews = encoded_reviews.tolist()
train_df['bert_embeddings'] = encoded_reviews

In [ ]:
def extract_user_profiles(df):
  """
  Extract item profiles by grouping by user id and taking mean review vector
  """
  user_groups = df.groupby('user_id')
  user_profiles = {}
  for usergrp in tqdm(user_groups):
      profiles = np.array([np.array(profile) for profile in np.array(usergrp[1]['bert_embeddings'])])
      mean_profile = np.mean(profiles,axis=0)
      user_profiles[usergrp[0]] = mean_profile
  return user_profiles

def extract_item_profiles(df):
  """
  Extract item profiles by grouping by business id and taking mean review vector
  """
  item_groups = df.groupby('business_id')
  item_profiles = {}
  for itemrgrp in tqdm(item_groups):
      profiles = np.array([np.array(profile) for profile in np.array(itemrgrp[1]['bert_embeddings'])])
      mean_profile = np.mean(profiles,axis=0)
      item_profiles[itemrgrp[0]] = mean_profile
  return item_profiles

In [ ]:
user_profiles = extract_user_profiles(df = train_df)
item_profiles = extract_item_profiles(df = train_df)

100%|██████████| 12739/12739 [00:14<00:00, 887.28it/s]


In [ ]:
print(f"Number of user profiles : {len(user_profiles)}")
print(f"Number of item profiles : {len(item_profiles)}")

Number of user profiles : 6395
Number of item profiles : 12739


In [ ]:
with open(os.path.join(data_dir, 'user_profiles_bert.pkl'), 'wb') as f:
  pickle.dump(user_profiles, f)

with open(os.path.join(data_dir, 'item_profiles_bert.pkl'), 'wb') as f:
  pickle.dump(item_profiles, f)